# Price Data Exploratory Analysis

This notebook explores historical cryptocurrency price data for the Sentinance platform.

## Contents
1. Data Loading
2. Price Distributions
3. Volatility Analysis
4. Correlation Analysis
5. Trend Detection

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime, timedelta

# Set plotting style
plt.style.use('dark_background')
sns.set_palette('husl')

print('Libraries loaded successfully!')

## 1. Data Loading

In [ ]:
# Define symbols to analyze
symbols = {
    'BTC': 'BTC-USD',
    'ETH': 'ETH-USD',
    'SOL': 'SOL-USD',
}

# Download 2 years of data
end_date = datetime.now()
start_date = end_date - timedelta(days=730)

data = {}
for name, ticker in symbols.items():
    print(f'Downloading {name}...')
    data[name] = yf.download(ticker, start=start_date, end=end_date, interval='1d')
    print(f'  {len(data[name])} records')

# Combine close prices
prices = pd.DataFrame({name: df['Close'] for name, df in data.items()})
prices.head()

## 2. Price Distributions

In [ ]:
# Daily returns
returns = prices.pct_change().dropna()

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, col in enumerate(returns.columns):
    axes[i].hist(returns[col], bins=50, alpha=0.7, edgecolor='white')
    axes[i].axvline(0, color='red', linestyle='--', alpha=0.5)
    axes[i].set_title(f'{col} Daily Returns Distribution')
    axes[i].set_xlabel('Return')

plt.tight_layout()
plt.savefig('price_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. Volatility Analysis

In [ ]:
# Rolling volatility (20-day)
volatility = returns.rolling(window=20).std() * np.sqrt(252)  # Annualized

fig, ax = plt.subplots(figsize=(14, 5))

for col in volatility.columns:
    ax.plot(volatility[col], label=col, alpha=0.8)

ax.set_title('20-Day Rolling Volatility (Annualized)')
ax.set_xlabel('Date')
ax.set_ylabel('Volatility')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Summary stats
print('\nVolatility Statistics:')
print(volatility.describe())

## 4. Correlation Analysis

In [ ]:
# Correlation matrix
corr_matrix = returns.corr()

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='RdYlGn', center=0, 
            square=True, linewidths=0.5, ax=ax)
ax.set_title('Return Correlation Matrix')

plt.tight_layout()
plt.savefig('correlation_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Trend Detection

In [ ]:
# Calculate trend indicators
btc = data['BTC'].copy()
btc['SMA_20'] = btc['Close'].rolling(20).mean()
btc['SMA_50'] = btc['Close'].rolling(50).mean()
btc['SMA_200'] = btc['Close'].rolling(200).mean()

# Plot
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(btc['Close'], label='BTC Price', alpha=0.9)
ax.plot(btc['SMA_20'], label='SMA 20', alpha=0.7)
ax.plot(btc['SMA_50'], label='SMA 50', alpha=0.7)
ax.plot(btc['SMA_200'], label='SMA 200', alpha=0.7)

ax.set_title('BTC Price with Moving Averages')
ax.set_xlabel('Date')
ax.set_ylabel('Price (USD)')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

Key findings from this analysis:
- BTC, ETH, and SOL show high positive correlation
- Volatility clusters during market stress periods
- Moving averages help identify trend direction